<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Task-Manager-agent" data-toc-modified-id="Task-Manager-agent-1">Task Manager agent</a></span><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1.1">Goal</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-1.2">Imports</a></span></li><li><span><a href="#Matching-the-viral-baseline" data-toc-modified-id="Matching-the-viral-baseline-1.3">Matching the viral baseline</a></span><ul class="toc-item"><li><span><a href="#Visualization" data-toc-modified-id="Visualization-1.3.1">Visualization</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-1.3.2">Evaluation</a></span></li></ul></li><li><span><a href="#Summary" data-toc-modified-id="Summary-1.4">Summary</a></span><ul class="toc-item"><li><span><a href="#Learnings" data-toc-modified-id="Learnings-1.4.1">Learnings</a></span></li></ul></li><li><span><a href="#TODO" data-toc-modified-id="TODO-1.5">TODO</a></span></li></ul></li></ul></div>

# Task Manager agent

## Goal

The goal is to implement a task manager bot. At the start of the turn the bot will assign different tasks to the units and later it will coordinate the actions based on their priority.

As a start point I will simply try to get a similar bot to `viral` and once I have that foundation I will start improving it.

## Imports

In [1]:
# Use this to reload changes in python scripts
%load_ext autoreload
%autoreload 2

In [2]:
import os
import random
import tempfile
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from kaggle_environments import make, evaluate
from functools import partial

from luxai.utils import render_game_in_html, create_temporal_python_file, set_random_seed
from luxai.agents.task_manager import TaskManagerAgent
from luxai.evaluation import play_matches_in_parallel, compute_result_ratios

Loading environment football failed: No module named 'gfootball'


In [3]:
plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (20, 5)  
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

## Matching the viral baseline

### Visualization

In [4]:
env = make("lux_ai_2021", debug=True, 
           configuration={'width': 12, 'height': 12, 'seed': 1, 
                          'episodeSteps': 41, 'annotations':True})
game_info = env.run(['simple_agent', TaskManagerAgent()])

In [4]:
set_random_seed(7)
env = make("lux_ai_2021", debug=True, 
           configuration={'width': 12, 'height': 12, 'seed': 0, 
                          'episodeSteps': 361, 'annotations':True})
game_info = env.run(['simple_agent', TaskManagerAgent()])
render_game_in_html(env)

Opening in existing browser session.


In [6]:
env = make("lux_ai_2021", debug=True, 
           configuration={'width': 12, 'height': 12, 'seed': 1, 
                          'episodeSteps': 361, 'annotations':True})
game_info = env.run(['../agents/viral.py', TaskManagerAgent()])
render_game_in_html(env)

Opening in existing browser session.


In [5]:
raise

RuntimeError: No active exception to reraise

### Evaluation

Create a temporal file with the agent and play against simple and viral agent.

In [4]:
text = """
from luxai.agents.task_manager import TaskManagerAgent

global_agent = TaskManagerAgent()

def agent(observation, configuration):
    return global_agent(observation, configuration)
"""
temporal_agent = create_temporal_python_file(text)

In [5]:
matches_results = play_matches_in_parallel(agents=[temporal_agent, 'simple_agent'])
compute_result_ratios(matches_results)

  0%|          | 0/100 [00:00<?, ?it/s]

(0.95, 0.0, 0.05)

In [6]:
matches_results = play_matches_in_parallel(agents=['../agents/viral.py', temporal_agent], n_matches=500)
matches_results += play_matches_in_parallel(agents=[temporal_agent, '../agents/viral.py'], n_matches=500)
compute_result_ratios(matches_results)

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

(0.47, 0.025, 0.505)

In [ ]:
(0.497, 0.028, 0.475)
(0.47, 0.025, 0.505) when setting the random seed

## Summary

### Learnings

## TODO

Function to create a temporal file with the agent